# High tip classification - Random forest

## Plain scikit-learn

**Hardware**: r5.8xlarge (32 CPU, 256 GB RAM)

In [1]:
from ml_utils import MLUtils

ml_utils = MLUtils(
    ml_task='high_tip',
    tool='scikit',
    model='random_forest',
)

# Load data and feature engineering

In [2]:
import numpy as np
import pandas as pd

In [3]:
%%time
tip_train = ml_utils.read_parquet_dir(f'{ml_utils.taxi_path}/data/ml/tip_train')
tip_train.shape

CPU times: user 2min 7s, sys: 1min 19s, total: 3min 26s
Wall time: 12min 11s


(219889897, 12)

In [4]:
tip_train.head()

,id,pickup_datetime,dropoff_datetime,pickup_taxizone_id,dropoff_taxizone_id,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,passenger_count,tip_fraction
0,a2acf0e861b84476b1ba8b91f08448f4,2016-07-10 06:56:05,2016-07-10 06:59:53,263.0,236.0,6,27,6,56,150,1.0,0.600000
1,bac9b63e96f14ca9a4e9c236d709f5f2,2016-07-10 10:50:18,2016-07-10 10:55:21,142.0,163.0,6,27,10,50,154,5.0,0.226667
2,6ec52187eb4a4ab6ae3ce632030c0322,2016-07-10 10:50:18,2016-07-10 11:08:38,74.0,66.0,6,27,10,50,154,1.0,0.000000
3,21102720ac044762977f91b30f8857a1,2016-07-10 10:50:19,2016-07-10 10:55:38,161.0,142.0,6,27,10,50,154,1.0,0.166667
4,3d002b5aa2404553adc0ad79c776bf32,2016-07-10 10:50:19,2016-07-10 11:08:54,226.0,261.0,6,27,10,50,154,1.0,0.206957


Training with the full data ran out of memory, so let's take a sample to be able to successfully train

In [5]:
sample = tip_train.sample(frac=0.3, replace=False, random_state=42)
sample.shape

(65966969, 12)

# Random forest classifier

We'll train a classifier than can predict "high-tip" rides - those where the tip percent is >25%

In [9]:
features = ml_utils.tip_vars.features
y_col = ml_utils.tip_vars.y_col
y_clf = ml_utils.tip_vars.y_clf
high_tip = ml_utils.tip_vars.high_tip

sample[y_clf] = (sample[y_col] > high_tip).astype(int)

In [10]:
sample[y_clf].value_counts()

0    48470272
1    17496697
Name: high_tip, dtype: int64

<br>

We don't need to one-hot encode or scale because we are using tree models. We can parallelize the training across all the cores in the single node using `n_jobs=-1`

In [11]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

In [12]:
%%time
with ml_utils.time_fit():
    _ = rfc.fit(sample[features], sample[y_clf])

CPU times: user 4h 3min 4s, sys: 4min 18s, total: 4h 7min 22s
Wall time: 9min 34s


## Save model

In [13]:
ml_utils.write_model(rfc)

uploading model to 's3://saturn-titan/nyc-taxi/ml_results/models/high_tip__scikit__random_forest.pkl'
successfully uploaded model


## Predict on test set

And calculate metrics. Save predictions and metrics to S3.

In [14]:
%%time

tip_test = ml_utils.read_parquet_dir(f'{ml_utils.taxi_path}/data/ml/tip_test')
tip_test[y_clf] = (tip_test[y_col] > high_tip).astype(int)

preds = tip_test[['id', y_clf]].copy()
preds.columns = ['id', 'actual']
preds['predicted'] = rfc.predict_proba(tip_test[features])[:, 1]

CPU times: user 3min 54s, sys: 43.1 s, total: 4min 37s
Wall time: 1min 38s


In [15]:
preds.head()

,id,actual,predicted
0,2e8f402e4dc44f2fae8b9328a237c4d2,0,0.212288
1,5f067a4121244f42bf460867c23b39c9,0,0.197804
2,60e8442d3d434df4959261905a279f55,0,0.209650
3,2d1537ce2ed347778e078eaee7eacd44,0,0.259579
4,13bb8a9ecbd04b559b7b9e40904026b0,0,0.253932


In [16]:
%%time
ml_utils.write_predictions(preds)

Writing predictions to 's3://saturn-titan/nyc-taxi/ml_results/predictions/high_tip__scikit__random_forest'
Done writing predictions
CPU times: user 10.1 s, sys: 2.36 s, total: 12.5 s
Wall time: 1min 17s


In [17]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(preds.actual, preds.predicted)
ml_utils.write_metric_df('auc', auc)

,ml_task,tool,model,metric,value,fit_seconds
0,high_tip,scikit,random_forest,auc,0.558524,574.561481
